[Completion Prompts Customization](https://docs.llamaindex.ai/en/stable/examples/customization/prompts/completion_prompts/#completion-prompts-customization)

In [2]:
MIXTRAL_TEXT_GENERATION_MODEL_REPO = "mistralai/Mixtral-8x7B-Instruct-v0.1"


import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


import torch

torch.cuda.is_available()
print("DEVICE:", "GPU" if torch.cuda.is_available() else "CPU")


from llama_index.core import Settings
from llama_index.embeddings.huggingface import (
    HuggingFaceEmbedding,
)
from llama_index.llms.huggingface import HuggingFaceInferenceAPI


Settings.embed_model = HuggingFaceEmbedding(cache_folder="./tmp/models/")

Settings.llm = HuggingFaceInferenceAPI(
    tokenizer_name=MIXTRAL_TEXT_GENERATION_MODEL_REPO,
    model_name=MIXTRAL_TEXT_GENERATION_MODEL_REPO,
    device_map="sequential",
)

DEVICE: GPU
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [3]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own.\n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)
refine_template = PromptTemplate(refine_template_str)

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/test/").load_data()
index = VectorStoreIndex.from_documents(documents)

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.83it/s]


In [5]:
print(index.as_query_engine().query("what is iris?"))

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.39it/s]



Iris is a wild flower found in the Himalayas from the west to the east, growing at heights of 2800-4000 m. on open slopes and grazing grounds. It has one of the loveliest names of all flowers and is characterized by its bright lilac-purple flowers with yellow-tipped beards or combs on the outer petals. The iris grows from an underground rhizome and likes wet ground. There are more than a dozen varieties of iris in the mountains, including I. germanica or Keor eka-mul, which is cultivated in the hills for orris oil, and I. nepalensis, or Chiluchi, whose roots are used in medicines as a diuretic and for bilious complaints.


In [6]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
    ).query("how to write in gujarati language ?")
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 21.09it/s]


To write in Gujarati language, you can use a Gujarati keyboard or a word processor with Gujarati language support. You can also use a virtual keyboard or a transliteration tool to type Gujarati characters using the English alphabet. Additionally, there are several Gujarati fonts available that you can use to write in Gujarati. It is important to note that Gujarati is written from right to left, unlike English which is written from left to right. Therefore, you will need to adjust your writing direction accordingly.
